# Case 1: Diagonal movement not allowed

### Dijkstra

In [9]:
#IMAGE Task_1_High.png
#Dijkstra with ONLY straight movement NO diagonal movement allowed

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img1 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img1)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img1 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img1[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.g = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img1[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img1[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def dijkstra(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minCost in enumerate(openList):
            if minCost.g < current.g:
                current = minCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img1[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img1[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img1[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g + 10

            # Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = dijkstra(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_d_nd = stop - begin 
print("Run time of program implementing Dijkstra is:",timeOfProgram_d_nd)

d_nd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        d_nd_cost = node.g

cv.imshow('dijkstra', img1)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_ds_result.png', img1)

Run time of program implementing Dijkstra is: 5.663496494293213


True

### Admissible

In [10]:
#IMAGE Task_1_High.png
#Astar with ONLY straight NO diagonal movement allowed
#ADMISSIBLE
#min of the absolute diff between x and y coordinates

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img2 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img2)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img2 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img2[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img2[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img2[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing A star algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img2[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img2[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img2[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g + 10
                
            #Record f and h:
            child.h = min(abs(child.coord[0] - end.coord[0]), abs(child.coord[1] - end.coord[1]))
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = astar(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_a_nd = stop - begin 
print("Run time of program implementing Astar with admissible heuristic is:",timeOfProgram_a_nd)

a_nd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        a_nd_cost = node.g
        
cv.imshow('Admissible', img2)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_admissible_nodiag_result.png', img2)

Run time of program implementing Astar with admissible heuristic is: 5.128916501998901


True

### Non- admissible

In [5]:
#IMAGE Task_1_High.png
#Astar with ONLY straight NO diagonal movement allowed
#HEURISTIC = Non admissible
#Square of distance between start and end coordinates

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img3 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img3)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img3 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img3[x, y]
    # Obstacles = rgb(255,255,255)
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:
    
    def __init__(self, parent=None, coord=None):
        self.coord = coord
        self.f = 0
        self.g = 0
        self.parent = parent
        
    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img3[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img3[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes    
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)
    
    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest fcost node and make it the current node
        current = openList[0]
        i = 0
        for j,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                i = j

        #Remove current from openList and add it to closedList
        openList.pop(i)
        closedList.append(current)

        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            path = []
            x = current.parent
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
                path = path[::-1]
            
            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img3[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112, 25, 25)
            
            img3[start.coord[0],start.coord[1]] = (113, 204, 45)
            img3[end.coord[0],end.coord[1]] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0)]:

            #Get adjacent nodes coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if (coord[0] > (999) or coord[0] < 0 or coord[1] > (999) or coord[1] < 0):
                continue
                
            #Node should be walkable - not obstacle
            if is_obstacle(coord[0],coord[1]):
                continue

            #Go to next node if this one is in closedList
            dummyVar = 0
            for node in closedList:
                if node.coord == coord:
                    dummyVar = 1
            if dummyVar == 1:
                continue
                
            child = MakeNode(current, coord)

            #Record gcost
            child.g = current.g + 10
                
            #Record f,h
            child.h = (child.coord[0] - end.coord[0]) ** 2 + (child.coord[1] - end.coord[1]) ** 2
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to openList
            openList.append(child)

            
#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(113, 204, 45)
endCoord = find_pixel_coord(60, 76, 231)

path = astar(startCoord, endCoord)
    
img3[startCoord[0]:(startCoord[0]+10),startCoord[1]:(startCoord[1]+10)] = (113, 204, 45)
img3[endCoord[0]:(endCoord[0]+10),endCoord[1]:(endCoord[1]+10)] = (60, 76, 231)
    
#Record run time of program
stop = time.time()
timeOfProgram_na_nd = stop - begin 
print("Run time of program implementing Astar with non admissible heuristic is:",timeOfProgram_na_nd)

na_nd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        na_nd_cost = node.g

cv.imshow('Non admissible', img3)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_nonadmis_nodiag_result.png', img3)

Run time of program implementing Astar with non admissible heuristic is: 3.9336037635803223


True

### Diagonal distance

In [6]:
#IMAGE Task_1_High.png
#Astar with ONLY straight NO diagonal movement allowed
#DIAGONAL DISTANCE

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img4 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img4)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img4 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img4[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img4[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img4[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing A star algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img4[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img4[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img4[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g + 10
                
            #Record f and h:
            child.h = max(abs(child.coord[0] - end.coord[0]), abs(child.coord[1] - end.coord[1]))
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = astar(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_dd_nd = stop - begin 
print("Run time of program implementing Astar with diagonal distance heuristic is:",timeOfProgram_dd_nd)

dd_nd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        dd_nd_cost = node.g

cv.imshow('Diagonal distance', img4)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_diagonal_nodiag_result.png', img4)

Run time of program implementing Astar with diagonal distance heuristic is: 4.396164655685425


True

### Manhattan distance

In [7]:
#IMAGE Task_1_High.png
#Astar with diagonal movement allowed
#MANHATTAN DISTANCE

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img5 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img5)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img5 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img5[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img5[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img5[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img5[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img5[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img5[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0), (-10, -10), (-10, 10), (10, -10), (10, 10)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14
                
            #Record f and h:
            child.h = abs(child.coord[0] - end.coord[0]) + abs(child.coord[1] - end.coord[1])
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = astar(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_md_nd = stop - begin 
print("Run time of program implementing Astar with manhattan heuristic is:",timeOfProgram_md_nd)

md_nd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        md_nd_cost = node.g

cv.imshow('Manhattan distance', img5)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_m_result.png', img5)

Run time of program implementing Astar with manhattan heuristic is: 2.497373104095459


True

### Euclidean distance

In [8]:
#IMAGE Task_1_High.png
#Astar with ONLY straight NO diagonal movement allowed
#HEURISTIC = Euclidean distance

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img6 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img6)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img6 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img6[x, y]
    # Obstacles = rgb(255,255,255)
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:
    
    def __init__(self, parent=None, coord=None):
        self.coord = coord
        self.f = 0
        self.g = 0
        self.parent = parent
        
    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img6[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img6[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes    
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)
    
    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest fcost node and make it the current node
        current = openList[0]
        i = 0
        for j,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                i = j

        #Remove current from openList and add it to closedList
        openList.pop(i)
        closedList.append(current)

        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            path = []
            x = current.parent
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
                path = path[::-1]
            
            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img6[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112, 25, 25)
            
            img6[start.coord[0],start.coord[1]] = (113, 204, 45)
            img6[end.coord[0],end.coord[1]] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0)]:

            #Get adjacent nodes coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if (coord[0] > (999) or coord[0] < 0 or coord[1] > (999) or coord[1] < 0):
                continue
                
            #Node should be walkable - not obstacle
            if is_obstacle(coord[0],coord[1]):
                continue

            #Go to next node if this one is in closedList
            dummyVar = 0
            for node in closedList:
                if node.coord == coord:
                    dummyVar = 1
            if dummyVar == 1:
                continue
                
            child = MakeNode(current, coord)

            #Record gcost
            child.g = current.g + 10
                
            #Record f,h
            child.h = np.sqrt((child.coord[0] - end.coord[0]) ** 2 + (child.coord[1] - end.coord[1]) ** 2)
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to openList
            openList.append(child)

            
#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(113, 204, 45)
endCoord = find_pixel_coord(60, 76, 231)

path = astar(startCoord, endCoord)
    
img6[startCoord[0]:(startCoord[0]+10),startCoord[1]:(startCoord[1]+10)] = (113, 204, 45)
img6[endCoord[0]:(endCoord[0]+10),endCoord[1]:(endCoord[1]+10)] = (60, 76, 231)
    
#Record run time of program
stop = time.time()
timeOfProgram_ed_nd = stop - begin 
print("Run time of program implementing Astar with euclidean heuristic is:",timeOfProgram_ed_nd)

ed_nd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        ed_nd_cost = node.g

cv.imshow('Euclidean', img6)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_euclidean_nodiag_result.png', img6)

Run time of program implementing Astar with euclidean heuristic is: 4.626990079879761


True

# Case 2: Diagonal movement allowed

### Dijkstra

In [13]:
#IMAGE Task_1_High.png
#Dijkstra with diagonal movement allowed

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img1 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img1)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img1 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img1[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.g = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img1[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img1[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def dijkstra(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minCost in enumerate(openList):
            if minCost.g < current.g:
                current = minCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img1[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img1[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img1[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0), (-10, -10), (-10, 10), (10, -10), (10, 10)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14

            # Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = dijkstra(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_d = stop - begin 
print("Run time of program implementing Dijkstra is:",timeOfProgram_d)

d_cost = 0
for node in closedList:
    if node.coord == endCoord:
        d_cost = node.g

cv.imshow('Dijkstra diagonal', img1)
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_result.png', img1)

Run time of program implementing Dijkstra is: 9.559300661087036


True

### Admissible heuristic

In [15]:
#IMAGE Task_1_High.png
#Astar with ONLY straight NO diagonal movement allowed
#ADMISSIBLE
#min of the absolute diff between x and y coordinates

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img2 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img2)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img2 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img2[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img2[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img2[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing A star algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img2[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img2[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img2[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0),(10, -10), (10, 10), (-10, 10), (-10, -10)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14
                
            #Record f and h:
            child.h = min(abs(child.coord[0] - end.coord[0]), abs(child.coord[1] - end.coord[1]))
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = astar(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_a = stop - begin 
print("Run time of program implementing Astar with admissible heuristic is:",timeOfProgram_a)

a_cost = 0
for node in closedList:
    if node.coord == endCoord:
        a_cost = node.g

cv.imshow('new_image', img2)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_admissible_result.png', img2)

Run time of program implementing Astar with admissible heuristic is: 11.353977680206299


True

### Non- admissible heuristic

In [17]:
#IMAGE Task_1_High.png
#Astar with diagonal movement allowed
#HEURISTIC = Non admissible
#DIAGONAL MOVEMENT ALLOWED

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img3 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img3)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img3 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img3[x, y]
    # Obstacles = rgb(255,255,255)
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:
    
    def __init__(self, parent=None, coord=None):
        self.coord = coord
        self.f = 0
        self.g = 0
        self.parent = parent
        
    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img3[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img3[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes    
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)
    
    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest fcost node and make it the current node
        current = openList[0]
        i = 0
        for j,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                i = j

        #Remove current from openList and add it to closedList
        openList.pop(i)
        closedList.append(current)

        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            path = []
            x = current.parent
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
                path = path[::-1]
            
            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img3[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112, 25, 25)
            
            img3[start.coord[0],start.coord[1]] = (113, 204, 45)
            img3[end.coord[0],end.coord[1]] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0), (-10, -10), (-10, 10), (10, -10), (10, 10)]:

            #Get adjacent nodes coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if (coord[0] > (999) or coord[0] < 0 or coord[1] > (999) or coord[1] < 0):
                continue
                
            #Node should be walkable - not obstacle
            if is_obstacle(coord[0],coord[1]):
                continue

            #Go to next node if this one is in closedList
            dummyVar = 0
            for node in closedList:
                if node.coord == coord:
                    dummyVar = 1
            if dummyVar == 1:
                continue
                
            child = MakeNode(current, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14
                
            #Record f,h
            child.h = (child.coord[0] - end.coord[0]) ** 2 + (child.coord[1] - end.coord[1]) ** 2
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to openList
            openList.append(child)

            
#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(113, 204, 45)
endCoord = find_pixel_coord(60, 76, 231)

path = astar(startCoord, endCoord)
    
img3[startCoord[0]:(startCoord[0]+10),startCoord[1]:(startCoord[1]+10)] = (113, 204, 45)
img3[endCoord[0]:(endCoord[0]+10),endCoord[1]:(endCoord[1]+10)] = (60, 76, 231)
    
#Record run time of program
stop = time.time()
timeOfProgram_na = stop - begin 
print("Run time of program implementing Astar with non admissible heuristic is:",timeOfProgram_na)

na_cost = 0
for node in closedList:
    if node.coord == endCoord:
        na_cost = node.g

cv.imshow('Non admissible', img3)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_nonadmis_result.png', img3)

Run time of program implementing Astar with non admissible heuristic is: 2.552792549133301


True

### Diagonal distance

In [18]:
#IMAGE Task_1_High.png
#Astar with diagonal movement allowed
#DIAGONAL DISTANCE

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img4 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img4)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img4 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img4[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img4[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img4[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing A star algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img4[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img4[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img4[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0), (-10, -10), (-10, 10), (10, -10), (10, 10)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14
                
            #Record f and h:
            child.h = max(abs(child.coord[0] - end.coord[0]), abs(child.coord[1] - end.coord[1]))
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = astar(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_dd = stop - begin 
print("Run time of program implementing Astar with diagonal distance heuristic is:",timeOfProgram_dd)

dd_cost = 0
for node in closedList:
    if node.coord == endCoord:
        dd_cost = node.g

cv.imshow('Diagonal distance', img4)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_diagonal_result.png', img4)

Run time of program implementing Astar with diagonal distance heuristic is: 4.3832762241363525


True

### Manhattan distance

In [19]:
#IMAGE Task_1_High.png
#Astar with diagonal movement allowed
#MANHATTAN DISTANCE

import time
import numpy as np
import cv2 as cv
__all__ = [cv]

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img5 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img5)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img5 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img5[x, y]
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:

    def __init__(self, parent, coord):
        self.parent = parent
        self.coord = coord
        self.f = 0
        self.g = 0
        self.h = 0

    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img5[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img5[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)

    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest gcost node and make it the current node
        current = openList[0]
        j = 0
        for i,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                j = i

        #Remove current from openList and add it to closedList
        openList.pop(j)
        closedList.append(current)


        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            x = current.parent
            path =[]
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
            path = path[::-1]

            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img5[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112,25,25)
            
            img5[start.coord[0]:(start.coord[0]+10),start.coord[1]:(start.coord[1]+10)] = (113, 204, 45)
            img5[end.coord[0]:(end.coord[0]+10),end.coord[1]:(end.coord[1]+10)] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0), (-10, -10), (-10, 10), (10, -10), (10, 10)]:

            #Get adjacent squares coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if coord[0] >= 1000 or coord[0] < 0 or coord[1] >= 1000 or coord[1] < 0:
                continue

            #Node should be walkable - not obstacle
            if is_obstacle(coord[0], coord[1]):
                continue

            #Go to next node if this one is in closedList
            check = 0
            for closedNode in closedList:
                if coord == closedNode.coord:
                    check = 1
                    break
            if check == 1:
                continue

            child = MakeNode(None, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14
                
            #Record f and h:
            child.h = abs(child.coord[0] - end.coord[0]) + abs(child.coord[1] - end.coord[1])
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to open list
            child.parent = current
            openList.append(child)

#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(45, 204, 113)
endCoord = find_pixel_coord(231, 76, 60)

path = astar(startCoord, endCoord)

#Record run time of program
stop = time.time()
timeOfProgram_md = stop - begin 
print("Run time of program implementing Astar with manhattan heuristic is:",timeOfProgram_md)

md_cost = 0
for node in closedList:
    if node.coord == endCoord:
        md_cost = node.g

cv.imshow('Manhattan', img5)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_m_result.png', img5)

Run time of program implementing Astar with manhattan heuristic is: 2.7199363708496094


True

### Euclidean distance

In [20]:
#IMAGE Task_1_High.png
#Astar with diagonal movement allowed
#HEURISTIC = EUCLIDEAN DISTANCE
#DIAGONAL MOVEMENT ALLOWED

#Start time
begin = time.time()

openList = []
closedList = []

#declare openList and closedList as global variables for storing the nodes that have been processed
img6 = cv.imread('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High.png', 1)

#Reading the image and displaying
cv.imshow('original image', img6)
cv.waitKey(0)
cv.destroyAllWindows()

# to find the start and end point
def find_pixel_coord(b, g, r):
    color = (b, g, r)
    loc = np.argwhere(img6 == color)
    x = (loc[0][0]).item()
    y = (loc[0][1]).item()
    return x, y

#If the node is an obstacle, return true otherwise false
#Obstacles = rgb(255,255,255)
def is_obstacle(x, y):
    b, g, r = img6[x, y]
    # Obstacles = rgb(255,255,255)
    if b == 255 and g == 255 and r == 255:
        return True
    else:
        return False

#Defines node as objects of Makenode class
#Attributes are parent, coord, g representing cost to reach the node from the start where cost = travel distance
class MakeNode:
    
    def __init__(self, parent=None, coord=None):
        self.coord = coord
        self.f = 0
        self.g = 0
        self.parent = parent
        
    def __eq__(self, other):
        return self.coord == other.coord

#To give nodes in openList a light blue colour
def colorOpen(l):
    for node in l:
        img6[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (255,144,30)

#To give nodes in closed list a yellow colour
def colorClosed(l):
    for node in l:
        img6[node.coord[0]:(node.coord[0] + 10), node.coord[1]:(node.coord[1] + 10)] = (0,215,255)

#Function to find path implementing Dijkstra algorithm
def astar(startCoord, endCoord):
    
    #Make start and end nodes    
    start = MakeNode(None, startCoord)
    end = MakeNode(None, endCoord)
    
    openList.append(start)
    current = MakeNode(start, startCoord)
    
    #Loop until no nodes in openList(no path)
    while len(openList) > 0:
        
        #Search lowest fcost node and make it the current node
        current = openList[0]
        i = 0
        for j,minfCost in enumerate(openList):
            if minfCost.f < current.f:
                current = minfCost
                i = j

        #Remove current from openList and add it to closedList
        openList.pop(i)
        closedList.append(current)

        #If we reached the goal node, backtrack the path by going to the parent of the previous node
        if current == end:
            path = []
            x = current.parent
            while x.parent is not None:
                path.append(x.coord)
                x = x.parent
                path = path[::-1]
            
            #Change bgr values of each node accordingly
            colorOpen(openList)
            colorClosed(closedList)
            for coord in path:
                img6[coord[0]:(coord[0] + 10), coord[1]:(coord[1] + 10)] = (112, 25, 25)
            
            img6[start.coord[0],start.coord[1]] = (113, 204, 45)
            img6[end.coord[0],end.coord[1]] = (60,76,231)            
            return path

        #Check adjacent squares from the steps - diagonal and straight movement
        for step in [(0, -10), (0, 10), (-10, 0), (10, 0), (-10, -10), (-10, 10), (10, -10), (10, 10)]:

            #Get adjacent nodes coordinates
            coord = (current.coord[0] + step[0], current.coord[1] + step[1])

            #Check if they are within range
            if (coord[0] > (999) or coord[0] < 0 or coord[1] > (999) or coord[1] < 0):
                continue
                
            #Node should be walkable - not obstacle
            if is_obstacle(coord[0],coord[1]):
                continue

            #Go to next node if this one is in closedList
            dummyVar = 0
            for node in closedList:
                if node.coord == coord:
                    dummyVar = 1
            if dummyVar == 1:
                continue
                
            child = MakeNode(current, coord)

            #Record gcost
            child.g = current.g
            if (step[0] * step[1]) == 0:
                child.g += 10
            else:
                child.g += 14
                
            #Record f,h
            child.h = np.sqrt((child.coord[0] - end.coord[0]) ** 2 + (child.coord[1] - end.coord[1]) ** 2)
            child.f = child.g + child.h

            #Check if it is already in openList - if it is then is this a better path?
            check = 0
            for node in openList:
                if coord == node.coord:
                    check = 1
                    if child.g < node.g:
                        node.g = child.g
                        node.f = child.f
                        node.parent = current
            if check == 1:
                continue

            #Add to openList
            openList.append(child)

            
#Find coordinates corresponding to start and goal nodes
startCoord = find_pixel_coord(113, 204, 45)
endCoord = find_pixel_coord(60, 76, 231)

path = astar(startCoord, endCoord)
    
img6[startCoord[0]:(startCoord[0]+10),startCoord[1]:(startCoord[1]+10)] = (113, 204, 45)
img6[endCoord[0]:(endCoord[0]+10),endCoord[1]:(endCoord[1]+10)] = (60, 76, 231)
    
#Record run time of program
stop = time.time()
timeOfProgram_ed = stop - begin 
print("Run time of program implementing Astar with euclidean heuristic is:",timeOfProgram_ed)

ed_cost = 0
for node in closedList:
    if node.coord == endCoord:
        ed_cost = node.g

cv.imshow('Euclidean', img6)
cv.waitKey(0)
cv.destroyAllWindows()
cv.imwrite('/home/ramona/Desktop/IIT KGP/AGV/Software/Task 1/Task_1_High_euclidean_result.png', img6)

Run time of program implementing Astar with euclidean heuristic is: 4.468373537063599


True

# Time taken to find path

In [ ]:
print("Case\t  Dijkstra\t  Admissible\t  Non Admissible\t  Diagonal\t  Manhattan\t  Euclidean\t")
print("Case 1 \t", round(timeOfProgram_d_nd,5),"\t", round(timeOfProgram_a_nd,5)," \t\t", round(timeOfProgram_na_nd,5)," \t", 
          round(timeOfProgram_dd_nd,5),"\t", round(timeOfProgram_md_nd,5),"\t", round(timeOfProgram_ed_nd,5))
print("Case 2\t", round(timeOfProgram_d,3),"\t\t", round(timeOfProgram_a,3),"\t\t\t", round(timeOfProgram_na,3),"\t\t",  round(timeOfProgram_dd,3),
           "\t\t",round(timeOfProgram_md,3),"\t\t", round(timeOfProgram_ed,3))

# Cost of path

In [ ]:
print("Case\t Dijkstra\t Admissible\t Non Admissible\t Diagonal\t Manhattan\t Euclidean\t")
print("Case 1\t", d_nd_cost,"\t\t", a_nd_cost,"\t\t", na_nd_cost,"\t\t", 
          dd_nd_cost,"\t\t", md_nd_cost,"\t\t", ed_nd_cost,"\t")
print("Case 2\t", d_cost,"\t\t", a_cost,"\t\t", na_cost,"\t\t",  dd_cost,
           "\t\t",md_cost,"\t\t", ed_cost)